In [ ]:
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
import math

arr_rate = 5
srv_rate = 2
duration = 3600*10
server_n = 3
t = 0
jobs_iat = []
while t <= duration:
    iat = np.random.exponential(1/arr_rate)
    t += iat
    jobs_iat.append(iat)
    
#Run the simulation
i = 0
waiting_times = []
response_times = []
queue = []

servers = [] #[service_start,end]
for s in range(server_n):
    servers.append([0,0])

busy_times = []
for s in range(server_n):
    busy_times.append([])

jump_times = [[0,0]]
queue_size = [[0,0]]
t = 0

def avl_server(t,servers):
    low = len(servers)
    i = 0
    for s in servers:
        if t > s[1]:
            if low == len(servers):
                low = i
            elif s[1] < servers[low][1]:
                low = i
        i += 1
    if low < len(servers):
        return low #return server with earliest idle time
    return len(servers)

def busy_server_n(t,servers):
    n = 0
    for s in servers:
        if t >= s[1]: #get empty servers
            n += 1
    return len(servers)-n #reverse

for job in jobs_iat:
    t += jobs_iat[i]
    st = np.random.exponential(1/srv_rate)
    #print(t)
    server_id = avl_server(t,servers) #get available server
    if server_id != len(servers) and len(queue) > 0: #there are available server, but there are jobs waiting in queue
        #fifoing the existing queue retroactively
        in_service = servers[server_id]
        while(server_id != len(servers) and len(queue) > 0):
            in_service = servers[server_id]
            popped = queue.pop(0)
            pop_st = popped[0]
            start_queue = popped[1]
            queue_size.append([in_service[1],len(queue)])
            waiting_times.append(in_service[1]-start_queue)
            print('Server '+str(server_id)+' is free and occupied again at '+ str(round(in_service[1],3)))
            print('Job (old) was served for '+str(round(in_service[1]-in_service[0],3)))
            servers[server_id] = [in_service[1], in_service[1]+pop_st]
            in_service = servers[server_id]
            busy_times[server_id].append(in_service)
            jump_times.append([in_service[0],busy_server_n(in_service[0],servers)])
            response_times.append(in_service[1]-start_queue)
            print('Job (new) was waiting in queue for '+str(round(in_service[0]-start_queue,3)))
            print('Queue size is now '+str(len(queue)))
            print('---------')
            server_id = avl_server(t,servers)
        #jump_times.append([in_service[0],busy_server_n(in_service[0],servers)])
    server_id = avl_server(t,servers) #get available server
    if server_id != len(servers) and len(queue) == 0: #there are (still) available servers and queue is also empty
        last_jump = jump_times[-1][0]
        tmp_jump = []
        sn = 0
        for s in servers:
            if t >= s[1] and s[1] > 0 and s[1] > last_jump:
                tmp_jump.append([s[1],busy_server_n(s[1],servers)])
                print('Server '+str(sn)+' is free at '+ str(round(s[1],3)))
                print('Job was served for '+str(round(s[1]-s[0],3)))
                print('Queue size is now '+str(len(queue)))
                print('---------')
            sn += 1
        tmp_jump.sort(key = lambda x: x[0]) 
        jump_times += tmp_jump
        #get in service
        in_service = servers[server_id]
        waiting_times.append(0)
        servers[server_id] = [t,t+st]
        in_service = servers[server_id]
        busy_times[server_id].append(in_service)
        jump_times.append([t,busy_server_n(t,servers)])
        response_times.append(st)
        print('Job arrived at: '+str(round(t,3)))
        print('Server '+str(server_id)+' is occupied at '+ str(round(t,3)))
        print('Job did not wait in queue')
        print('---------')
    else: #server is busy, regardless of queue
        queue.append([st,t])
        queue_size.append([t,len(queue)])
        print('Job arrived at: '+str(round(t,3)))
        print('Job is put in queue')
        print('Queue size is now '+str(len(queue)))
        print('---------')
    i += 1



In [ ]:
#Plot the jump times
start_t = 0
end_t = 30

tmp_jump = []
tmp_queue = []
init_jt = -1
for j in jump_times:
    if j[0] < start_t:
        init_jt += 1
        continue
    if j[0] > end_t:
        break
    tmp_jump.append(j)
if init_jt >= 0:
    tmp_jump = [[start_t,jump_times[init_jt][1]]] + tmp_jump
tmp_jump.append([end_t,tmp_jump[-1][1]])

init_qs = -1
for q in queue_size:
    if q[0] < start_t:
        init_qs += 1
        continue
    if q[0] > end_t:
        break
    tmp_queue.append(q)
if init_qs >= 0:
    tmp_queue = [[start_t,queue_size[init_qs][1]]] + tmp_queue
tmp_queue.append([end_t,tmp_queue[-1][1]])

plt.figure(figsize=(17,5))
plt.step([q[0] for q in tmp_jump], [q[1] for q in tmp_jump], 'r', where='post', label = 'Server busy')
plt.step([q[0] for q in tmp_queue], [q[1] for q in tmp_queue], 'b', where='post', label = 'Queue size')
plt.legend()
plt.show()

#Plot the waiting and response times
avg_wait = []
n = 1
tot = 0
for w in waiting_times:
    tot += w
    avg_wait.append(tot/n)
    n += 1

avg_resp = []
n = 1
tot = 0
for w in response_times:
    tot += w
    avg_resp.append(tot/n)
    n += 1

plt.figure(figsize=(15,10))
plt.plot([i for i in range(len(avg_wait))], avg_wait, label = "Avg Wait Times")
plt.plot([i for i in range(len(avg_resp))], avg_resp, label = "Avg Response Times")
plt.legend()
plt.show()

In [ ]:
def erlang_c(m,lamb,mu):
    ro = lamb/(m*mu)
    last = 0
    for k in range(m):
        up = pow((m*ro),k)
        last += (up/math.factorial(k))
    mid = math.factorial(m)/(pow((m*ro),m))
    down = 1 + ((1-ro)*mid*last)
    return 1/down

exp_resp = (erlang_c(server_n,arr_rate,srv_rate)/((server_n*srv_rate)-arr_rate))+(1/srv_rate)
print(mean(waiting_times))
print(mean(response_times))
print('------')
print(exp_resp)